# Projecting Indianapolis Crime Data onto Interactive Maps

__SUMMARY__:

The Indianapolis Metro Police Department (IMPD) published incident data on <a href="http://www.indy.gov/eGov/City/DPS/IMPD/Crimes/Pages/UCRDownload.aspx">indy.gov</a> for 2014, 2015, and 2016 - for various types of incidents: homocides, assult, burglery, stolen vehicles, etc. Relevent to this experiment, each incident noted the police beat, department id, crime, address, and GPS coordinates. 

My previous notebook (**"Deep Dive into Indianapolis Crime Data"**) matched 97% of the GPS coordinates within 200 feet of their corresponding addresses - probably because  police cars parked at the crime scene when logging their coordinates, and specified the most relevant addresses for administrative procedures. 

Unfortunately, it appears that IMPD only started logging in their GPS Coordinates at the start of 2014. Some months in 2015 and 2015 are completely missing. I'm not sure if this is new technology or if they utilized a different coordinate system.

__PURPOSE__:
- How can we use Python to generate interactive maps?

__GOALS__:
-  What is the busiest police beat?

-  Forcast the areas of Indianapolis with the highest liklihood of stolen vehicles as if its a weather map



***__SCROLL TO THE BOTTOM OF THE PAGE TO SEE MAPS__***
________________________


### Load the libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pyproj import Proj, transform

import folium
import folium.plugins as plugins

from json import dumps, load

#SOURCE OF DATA http://www.indy.gov/eGov/City/DPS/IMPD/Crimes/Pages/UCRDownload.aspx

#By Grayson Swaim - goswaim100@gmail.com

### Understand the Coordinate System 

The coordinate system for the IPS incident data spreadhseet is based on this state plane: 
NAD_1983_StatePlane_Indiana_East_FIPS_1301_Feet

I refered to this website to verify the accuracy of the data:
http://www.earthpoint.us/StatePlane.aspx
Select 1301 Indiana East, using the US Survey in Feet

ALSO READ THIS TO UNDERSTAND HOW TO HANDLE COORDINATE DATA IN ORDER TO PROJECT THEM ONTO A MAP:
http://downloads2.esri.com/support/documentation/ao_/710Understanding_Map_Projections.pdf

---------------------------------------------------------------------------------------------------------

In [2]:
#inProj: input coordinate system
#outProj: output coordinate system (the basic lat log)
#This will be fed into the transform function and ultimate spit out the output long lat degrees.
#Not entirely sure why the parameter "preserve_units" must be true. I stumbled upon this in pure luck.
#Look through pyproj\data folder in APPDATA to discover init parameters. I just searched for FIPS 1301 
#and discovered the comment: "NAD 1983 StatePlane Indiana East FIPS 1301 Feet"

inProj = Proj(init='esri:102673', preserve_units=True)
outProj = Proj(init='epsg:4326')

#Convert the FIPS 1301 coordinates into the conventional coordinate system.
def get_long_lat(x):
    y_coord, x_coord = transform(inProj, outProj, x[0], x[1], radians=False)
    return x_coord, y_coord

### Read the Data

In [3]:
columnspecs = [(0,5), (5, 33), (33, 44), (44, 51), (51, 63), (63, 108), (108, 115), (115, 137), (137,159)]

i2014 = pd.read_fwf("resources/2014_incidents.txt", 
                    skiprows=1, 
                    names=['UCR','CRIME','DATE','TIME','CASE','ADDRESS','BEAT','X_COORD','Y_COORD'],
                    colspecs=columnspecs)

i2015 = pd.read_fwf("resources/2015_incidents.txt", 
                    skiprows=1, 
                    names=['UCR','CRIME','DATE','TIME','CASE','ADDRESS','BEAT','X_COORD','Y_COORD'],
                    colspecs=columnspecs)

### Merge multiple years of data into one set

In [4]:
i2016 = pd.read_csv("resources/2016_incidents.csv")
i2016 = i2016[i2016.columns.drop(['ID', 'OBJECTID'])]
i2016.columns = i2015.columns

all(i2014.dtypes == i2015.dtypes) == all(i2015.dtypes == i2016.dtypes)

True

In [5]:
df = i2014.append(i2015).copy()
df = df.append(i2016).copy()
df['GPS_COORD_X'], df['GPS_COORD_Y'] = zip(*df[['X_COORD', 'Y_COORD']].apply(get_long_lat, axis=1))

#Erase
i2014 = None
i2015 = None
i2016 = None

### Establish coordinate boundaries for Indianapolis and eliminate outliers

In [6]:
#NORTHEAST BOUNDARY
NE_B = (39.965782, -86.014354)
#SOUTHWEST BOUNDARY
SW_B = (39.638192, -86.362483)

#Found erroneous data points where crimes occurring 
#in Indianapolis contained coordinates for the Phillipines or other Asian countries. 
#Eliminate those outliers 
df = df[df['GPS_COORD_X'].between(SW_B[0], NE_B[0])]
df = df.reset_index(drop=True)
df = df[df['GPS_COORD_Y'].between(SW_B[1], NE_B[1])]
df = df.reset_index(drop=True)

Crediting GEOJSON and TOPOJSON files to http://mapshaper.org/

### Load data onto a Folium Map Object

In [7]:
m = folium.Map(location=[df.GPS_COORD_X.mean(), df.GPS_COORD_Y.mean()], tiles='Stamen Toner', zoom_start=11)

folium.GeoJson(load(open("resources/shapefiles/Indianapolis_Police_Zones/geo_police_zones.json")), name='geojson').add_to(m)

folium.TopoJson(open("resources/shapefiles/Indianapolis_Police_Zones/topo_police_zones.json"),
                "objects.Indianapolis_Police_Zones",
                name='topojson',
               ).add_to(m)

folium.LayerControl().add_to(m)

### What is the busiest police beat to work on? ( Warning: some districts may contain multiple police beats)

In [8]:
beat_incident_count = df[['BEAT', 'CASE']].groupby(by='BEAT', as_index=False).agg(np.count_nonzero)
beat_incident_count.columns = ['BEAT', 'CASE_COUNT']

In [9]:
def getmap(beat_map_breakdown):
    m = folium.Map(location=[df.GPS_COORD_X.mean(), df.GPS_COORD_Y.mean()], tiles='Stamen Toner', zoom_start=11)
    m.choropleth(
        load(open("resources/shapefiles/Indianapolis_Police_Zones/geo_police_zones.json")),
        data=beat_incident_count,
        columns=['BEAT', 'CASE_COUNT'],
        key_on='properties.POLICEZONE',
        fill_color='YlOrRd',
        fill_opacity=0.3,
        highlight=True,
        threshold_scale=list(np.linspace(beat_map_breakdown['CASE_COUNT'].min(), 
                                         beat_map_breakdown['CASE_COUNT'].max(), 
                                         num=4))
        )

    gdf = gpd.read_file("resources/shapefiles/Indianapolis_Police_Zones/geo_police_zones.json")
    gdf['X'] = gdf['geometry'].apply(lambda x: x.centroid.coords.xy[1][0])
    gdf['Y'] = gdf['geometry'].apply(lambda x: x.centroid.coords.xy[0][0])
    idx_chg = gdf[gdf['POLICEZONE']=='Excluded']['POLICEZONE'].index
    gdf.loc[idx_chg, 'POLICEZONE'] = gdf.loc[idx_chg]['JURISDCTN'] + "_" + gdf.loc[idx_chg]['OBJECTID'].astype(str)

    x = gdf.iterrows()
    print(x)
    while True:
        try:
            c = next(x)
            folium.Marker(location=[c[1]['X'], c[1]['Y']], popup=c[1]['POLICEZONE']).add_to(m)
        except:
            break
    return m

## Busiest Police Beats
#### The more incidents that occur in a given district, the more red it will appear. Overall, ND70 appears to be the busiest police beat.

In [10]:
beat_incident_count = df[['BEAT', 'CASE']].groupby(by='BEAT', as_index=False).agg(np.count_nonzero)
beat_incident_count.columns = ['BEAT', 'CASE_COUNT']

m = getmap(beat_incident_count)
m.save("html/beat_map.html")
m

<generator object DataFrame.iterrows at 0x115898410>


## Project the highest frequency of stolen vehicles onto an Interactive Map

In [11]:
df['DATE'] = pd.to_datetime(df['DATE'],)
sln_vh = df[df['CRIME']=='STOLEN VEHICLE']
sln_vh = sln_vh[['DATE','GPS_COORD_X', 'GPS_COORD_Y', 'CRIME']]
sln_vh['YEAR'] = sln_vh['DATE'].apply(lambda x: x.year)
sln_vh['MONTH'] = sln_vh['DATE'].apply(lambda x: x.month)
sln_vh['MONTH_YEAR'] = sln_vh['YEAR'].astype(str) + "-" + sln_vh['MONTH'].astype(str) + "-1"
sln_vh['COORDS'] = sln_vh[['GPS_COORD_X', 'GPS_COORD_Y']].apply(lambda x: np.array(x), axis=1).values.tolist()

#Build each time period

sln_vh['MONTH_YEAR'] = pd.to_datetime(sln_vh['MONTH_YEAR'])
heat_data_year = [[[row['GPS_COORD_X'],row['GPS_COORD_Y']] for index, row in sln_vh[sln_vh['YEAR']==i].iterrows()] for i in sln_vh['YEAR'].unique()]
heat_data_month = [[[row['GPS_COORD_X'],row['GPS_COORD_Y']] for index, row in sln_vh[sln_vh['MONTH_YEAR']==i].iterrows()] for i in sln_vh['MONTH_YEAR'].unique()]
heat_data_date = [[[row['GPS_COORD_X'],row['GPS_COORD_Y']] for index, row in sln_vh[sln_vh['DATE']==i].iterrows()] for i in sln_vh['DATE'].unique()]

#### Interactive map. Select the "Play" button to see the trend of stolen vehicles by month.

In [12]:
m = folium.Map(location=[df.GPS_COORD_X.mean(), df.GPS_COORD_Y.mean()], tiles='Stamen Toner', zoom_start=11)
idx = pd.MultiIndex(levels=[sln_vh['MONTH_YEAR'].unique()], 
                    labels=[np.arange(sln_vh['MONTH_YEAR'].nunique())])

hm = plugins.HeatMapWithTime(
    heat_data_month,
    auto_play=True,
    max_opacity=0.3
).add_to(m)
m.save("html/timeheatmap_carsstolen.html")

m

### In 2016, it appears the greatest density of stolen cars occur on the south east side of indianapolis